## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import models, datasets, transforms
from sklearn.model_selection import StratifiedKFold
import numpy as np
from utils import train_model, evaluate_model
import random

## Set seed for reproducibility

In [2]:
def set_seed(seed):
    """
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed to be 42
set_seed(42)

## Resnet-18

In [3]:
# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the resnet model to use
def create_resnet_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

## Helper Functions

In [4]:
import itertools
import torch
import numpy as np

def load_dataset(dataset_folder):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.436, 0.385, 0.344], std=[0.296, 0.269, 0.261])
    ])
    
    dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)
    return dataset
    
def grid_search_tuning(dataset, num_classes, learning_rate, batch_sizes, num_epochs=5):
    best_accuracy = 0
    best_f1 = 0
    best_params = {}

    for lr, bs in itertools.product(learning_rates, batch_sizes):
        print(f"Testing LR: {lr}, BS: {bs}")

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        targets = np.array([dataset.targets[i] for i in range(len(dataset))])
        fold_accuracies = []
        fold_f1_scores = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(dataset)), targets)):
            print(f'Fold {fold + 1}')
            
            train_sampler = Subset(dataset, train_idx)
            val_sampler = Subset(dataset, val_idx)
            
            train_dataloader = DataLoader(train_sampler, batch_size=bs, shuffle=True)
            val_dataloader = DataLoader(val_sampler, batch_size=bs, shuffle=False)
            
            model = create_resnet_model(num_classes)
            trained_model = train_model(model, train_dataloader, val_dataloader, num_epochs=num_epochs, lr=lr)
            
            fold_accuracy, fold_f1 = evaluate_model(trained_model, val_dataloader)
            fold_accuracies.append(fold_accuracy)
            fold_f1_scores.append(fold_f1)
            print(f'Fold {fold + 1} Accuracy: {fold_accuracy:.4f}, F1 Score: {fold_f1:.4f}')
        
        mean_accuracy = np.mean(fold_accuracies)
        mean_f1 = np.mean(fold_f1_scores)
        print(f'Mean Accuracy for LR: {lr}, BS: {bs}: {mean_accuracy:.4f}')
        print(f'Mean F1 Score for LR: {lr}, BS: {bs}: {mean_f1:.4f}')

        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_f1 = mean_f1
            best_params = {'learning_rate': lr, 'batch_size': bs}
    
    print(f'Best Parameters: {best_params}')
    print(f'Best Accuracy: {best_accuracy:.4f}')
    print(f'Best F1 Score: {best_f1:.4f}')

## Start the tuning

In [5]:
# Set up device and dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_folder = '../data/lfw'
dataset = load_dataset(dataset_folder)
num_classes = len(dataset.classes)

In [5]:
# Define ranges for hyperparameters
learning_rates = [0.01, 0.1]
batch_sizes = [32, 64, 16]

# Perform grid search tuning
grid_search_tuning(dataset, num_classes, learning_rates, batch_sizes, num_epochs=50)

Testing LR: 0.01, BS: 16
Fold 1
Epoch 1/50, Train Loss: 3.0894, Val Loss: 3.0219
Epoch 2/50, Train Loss: 2.3778, Val Loss: 2.3575
Epoch 3/50, Train Loss: 2.2964, Val Loss: 2.2772
Epoch 4/50, Train Loss: 2.2796, Val Loss: 2.4654
Epoch 5/50, Train Loss: 2.2761, Val Loss: 2.3433
Epoch 6/50, Train Loss: 2.2602, Val Loss: 2.2309
Epoch 7/50, Train Loss: 2.2119, Val Loss: 2.2540
Epoch 8/50, Train Loss: 2.1428, Val Loss: 2.2443
Epoch 9/50, Train Loss: 2.0086, Val Loss: 2.0876
Epoch 10/50, Train Loss: 1.8856, Val Loss: 2.3249
Epoch 11/50, Train Loss: 1.7557, Val Loss: 1.9860
Epoch 12/50, Train Loss: 1.6764, Val Loss: 2.1116
Epoch 13/50, Train Loss: 1.5894, Val Loss: 1.8620
Epoch 14/50, Train Loss: 1.4350, Val Loss: 1.6537
Epoch 15/50, Train Loss: 1.3154, Val Loss: 2.2354
Epoch 16/50, Train Loss: 1.1407, Val Loss: 1.6521
Epoch 17/50, Train Loss: 1.0637, Val Loss: 1.5283
Epoch 18/50, Train Loss: 0.7718, Val Loss: 1.3036
Epoch 19/50, Train Loss: 0.6941, Val Loss: 1.3520
Epoch 20/50, Train Loss: 0.

# Refined Tuning
Try even smaller batch sizes.

In [6]:
# Define ranges for hyperparameters
learning_rates = [0.01]
batch_sizes = [4, 8, 16]

# Perform grid search tuning
grid_search_tuning(dataset, num_classes, learning_rates, batch_sizes, num_epochs=50)

Testing LR: 0.01, BS: 4
Fold 1
Epoch 1/50, Train Loss: 2.6324, Val Loss: 2.3669
Epoch 2/50, Train Loss: 2.2805, Val Loss: 2.8902
Epoch 3/50, Train Loss: 2.2609, Val Loss: 2.3232
Epoch 4/50, Train Loss: 2.2663, Val Loss: 2.1397
Epoch 5/50, Train Loss: 2.2087, Val Loss: 2.2571
Epoch 6/50, Train Loss: 2.0892, Val Loss: 2.5202
Epoch 7/50, Train Loss: 2.0420, Val Loss: 2.0671
Epoch 8/50, Train Loss: 1.9505, Val Loss: 1.8806
Epoch 9/50, Train Loss: 1.8371, Val Loss: 1.7282
Epoch 10/50, Train Loss: 1.7503, Val Loss: 1.6227
Epoch 11/50, Train Loss: 1.6511, Val Loss: 1.6376
Epoch 12/50, Train Loss: 1.5404, Val Loss: 1.9223
Epoch 13/50, Train Loss: 1.4631, Val Loss: 1.3568
Epoch 14/50, Train Loss: 1.2683, Val Loss: 1.2266
Epoch 15/50, Train Loss: 1.1074, Val Loss: 1.8480
Epoch 16/50, Train Loss: 0.9087, Val Loss: 1.1263
Epoch 17/50, Train Loss: 0.7231, Val Loss: 1.1452
Epoch 18/50, Train Loss: 0.5518, Val Loss: 1.0412
Epoch 19/50, Train Loss: 0.4994, Val Loss: 1.3491
Epoch 20/50, Train Loss: 0.3

## Results

```
Best Parameters: {'learning_rate': 0.01, 'batch_size': 16}
```